<a href="https://colab.research.google.com/github/NiniLaNouille/testing-gtm-implementation/blob/main/data_warehouses_connection_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Warehouses Connection example
This is an example on how to connect to our snowflake and BigQuery instance from a Google Colab notebook.

In [ ]:
# Install required packages
!pip install snowflake-connector-python
!pip install google-cloud-bigquery

Imports necessary libraries

In [ ]:
from google.colab import userdata
import snowflake.connector
from google.colab import auth
from google.cloud import bigquery
import pandas as pd

In [ ]:
# Snowflake configuration
conn = snowflake.connector.connect(
    user='cloudfunctions',
    password=userdata.get('SNOWFLAKE_PASSWORD'),
    account='ju81100.europe-west4.gcp',
    warehouse='reporting',
    database='analytics',
    schema='dbt'
)

# Create a cursor object
cur = conn.cursor()

In [ ]:
# BigQuery configuration
auth.authenticate_user()
client = bigquery.Client()

Example of an execution of a query in Snowflake.

In [ ]:
# Execute a query
cur.execute("SELECT * FROM fct_order_row LIMIT 10")

# Fetch the results
results = cur.fetchall()

# Display the results
for row in results:
    print(row)

# Close the cursor and the connection
cur.close()
conn.close()

(13676827, 'europe-north1', 'FI', 'FI', 1428076, 417, None, 5, 1, 237600, 'jNm81MIKV5MH8Qo56N4g1KDpb8G3', 19822, 'Kantolan Rapea pallopipari valkosuklaa 150g', None, datetime.date(2024, 6, 9), Decimal('0.690000'), Decimal('2.390000'), Decimal('-0.100000'), None, '[FI] Standard pricing', 'procurement campaign', Decimal('1.000000'), Decimal('0.140000'), Decimal('-0.100000'), Decimal('0.610000'), Decimal('0.690000'), 'Ax6gInSPEHV0c0UDYOPE2U1N3gbzHYQGKR9xTtBEFcM=', 'RWyGfnCGQNrGool5MJUH2QGvVWhAIMKTYG7XOgHF3hc=', 'PDORTEwzP+kMC5J0QxtpZT9vdHh21Uqt5kGqiHTVcJw=', 'iyes+KHsKs4IySTHZOud3lEFGHkoT12Z6TqKpooyz/I=', 'mXc/EUAAn4ztilHKlhXXrRbdlFjGYQzJtZd2jKHeNls=', '33450', '', 'FI', '4oc2kt95EkboCqpe5nAjK+SESya9nDkL4uhVjLAmywk=', 'Siivikkala', False, 'Ax6gInSPEHV0c0UDYOPE2U1N3gbzHYQGKR9xTtBEFcM=', 'RWyGfnCGQNrGool5MJUH2QGvVWhAIMKTYG7XOgHF3hc=', 'PDORTEwzP+kMC5J0QxtpZT9vdHh21Uqt5kGqiHTVcJw=', '33450', 'mXc/EUAAn4ztilHKlhXXrRbdlFjGYQzJtZd2jKHeNls=', 'Siivikkala', '4oc2kt95EkboCqpe5nAjK+SESya9nDkL4uhVjL

Example of a query executed in BigQuery

In [ ]:
query = """
    SELECT
      *
    FROM `bigquery-public-data.usa_names.usa_1910_2013`
    LIMIT 10
"""

# Execute the query and convert the results to a DataFrame
df = client.query(query).to_dataframe()

# Display the DataFrame
df.head()

NotFound: 404 POST https://bigquery.googleapis.com/bigquery/v2/projects//jobs?prettyPrint=false: Request couldn't be served.

Location: None
Job ID: 27e325b6-bf01-4137-bde1-5cd043da58db


# Data extract from Snowflake

The query is a bit long due to the needs of differents table to be connected. In that case we are using a script to get all at the same place.

In [ ]:
# Install required packages
!pip install snowflake-connector-python
!pip install google-cloud-bigquery

In [ ]:
# Execute a query
cur.executescript("""with calendar_complet as (SELECT n, theDate,
  DECODE (EXTRACT('dayofweek',theDate),
    1 , 'Monday',
    2 , 'Tuesday',
    3 , 'Wednesday',
    4 , 'Thursday',
    5 , 'Friday',
    6 , 'Saturday',
    0 , 'Sunday'
    ) theDayOfTheWeek,
DECODE (EXTRACT(month FROM theDate),
    1 , 'January',
    2 , 'February',
    3 , 'March',
    4 , 'April',
    5 , 'May',
    6 , 'June',
    7 , 'July',
    8 , 'August',
    9 , 'september',
    10, 'October',
    11, 'November',
    12, 'December'
    ) theMonth,
  EXTRACT(month FROM theDate) as theMonthNum,
  EXTRACT(year from theDate) theYear
  FROM
  (SELECT
    row_number() OVER (order by seq4()) AS n,
    DATEADD(day, row_number() OVER (order by seq4())-1, TO_DATE('2023-01-01'))
    AS theDate
    FROM table(generator(rowCount => (365 + 122))
    ))  ORDER BY n ASC),
calendar_ref as(
SELECT
    DISTINCT theYear,
    theMonth,
    theMonthNum
from calendar_complet
),
customer_list_ref as (
SELECT
REGION, CUSTOMER_INFORMATION_EMAIL,CUSTOMER_ID
FROM ANALYTICS.DBT.DIM_CUSTOMER
where CUSTOMER_ID is not null
),
reference_customer as (SELECT
REGION,
CUSTOMER_INFORMATION_EMAIL,
CUSTOMER_ID,
theMonth,
theMonthNum,
theYear
fROM customer_list_ref CROSS JOIN calendar_ref
ORDER BY theYear,theMonthNum,theMonth),

order_aggregate as (SELECT
REGION,
CUSTOMER_INFORMATION_EMAIL,
CUSTOMER_ID,
COUNT( ORDER_ID) as NUMBER_ORDER_TOTAL
FROM ANALYTICS.DBT.FCT_ORDER
GROUP BY REGION, CUSTOMER_INFORMATION_EMAIL,CUSTOMER_ID),

reference_list as (SELECT
r.REGION,
r.CUSTOMER_INFORMATION_EMAIL,
r.CUSTOMER_ID,
theMonth,
theMonthNum,
theYear,
NUMBER_ORDER_TOTAL
fROM reference_customer r CROSS JOIN order_aggregate a WHERE r.CUSTOMER_ID=a.CUSTOMER_ID AND r.REGION=a.REGION AND r.CUSTOMER_INFORMATION_EMAIL=a.CUSTOMER_INFORMATION_EMAIL
ORDER BY theYear,theMonthNum,theMonth),

ces_last_time as (
SELECT  CUSTOMER_ID, MAX(publish_time) as last_time
FROM ANALYTICS.DBT.FCT_CES_RESPONSE
group by CUSTOMER_ID
),

 ces_last_score as (
SELECT  f.CUSTOMER_ID, score as last_CES
FROM ANALYTICS.DBT.FCT_CES_RESPONSE f inner join ces_last_time l on f.CUSTOMER_ID=l.CUSTOMER_ID and f.publish_time=l.last_time
),

ces_average_score as(
SELECT  CUSTOMER_ID, AVG(score) as CES_AVG_score
FROM ANALYTICS.DBT.FCT_CES_RESPONSE
group by CUSTOMER_ID
),

ces_data as (select
a.CUSTOMER_ID, CES_AVG_score, last_CES
from ces_average_score a inner join ces_last_score l on a.CUSTOMER_ID=l.CUSTOMER_ID
order by CUSTOMER_ID),

status as (SELECt
REGION, CUSTOMER_INFORMATION_EMAIL, STATUS
FROM ANALYTICS.DBT_REPORTS.CUSTOMER_STATUS_HISTORY
where IS_CURRENT=TRUE),

customer_row as (SELECT
c.REGION,
c.CUSTOMER_INFORMATION_EMAIL,
c.CUSTOMER_ID,
DATEDIFF(day,LATEST_ORDER_AT, current_timestamp) as number_days_since_last_order,
AVERAGE_ORDER_TOTAL_INCLUDING_VAT,
LIFETIME_DISCOUNT_VALUE_VAT0,
LIFETIME_NET_SALES_VAT0,
DAYS_TO_REPURCHASE,
DAYS_SINCE_FIRST_ORDER,
STATUS,
ORDER_COUNT
FROM ANALYTICS.DBT.DIM_CUSTOMER c LEFT OUTER JOIN status s ON s.REGION=c.REGION AND s.CUSTOMER_INFORMATION_EMAIL=c.CUSTOMER_INFORMATION_EMAIL),

ga4_per_month as ( SELECT
REGION,
YEAR(SESSION_DATE) as SESSION_YEAR_PERIOD,
MONTH(SESSION_DATE) as SESSION_MONTH_PERIOD,
CUSTOMER_ID,
DATEDIFF(day, MAX(SESSION_DATE), current_timestamp) as number_days_since_last_session,
count(GA_SESSION_ID) as sessions,
count(case when engaged_session then 1 end) as engaged_sessions,
count(case when quality_session then 1 end) as quality_sessions,
count(case when cart_addition_sessions  then 1 end) as cart_addition_sessions,
count(case when begin_checkout_sessions then 1 end) as begin_checkout_sessions,
count(case when subscribers_sessions then 1 when sms_subscribers_sessions then 1 end) as subscribers_sessions,
count(case when reached_20e_in_basket then 1 end) as reached_20e_in_basket_sessions,
count(case when reached_35e_in_basket then 1 end) as reached_35e_in_basket_sessions,
count(case when reached_5th_product_in_cart then 1 end ) as reached_5th_product_in_cart_sessions
FROM ANALYTICS.DBT.FCT_GA_SESSION
GROUP BY  REGION,CUSTOMER_ID, YEAR(SESSION_DATE), MONTH(SESSION_DATE)),



ga4_total as (select
REGION,
CUSTOMER_ID,
MIN(number_days_since_last_session) as number_days_since_last_session,
SUM(sessions) as total_sessions,
SUM(engaged_sessions) as total_engaged_sessions,
SUM(quality_sessions) as total_quality_sessions,
SUM(cart_addition_sessions) as total_cart_addition_sessions,
SUM(begin_checkout_sessions) as total_begin_checkout_sessions,
SUM(subscribers_sessions) as total_subscribers_sessions,
SUM(reached_20e_in_basket_sessions) as total_reached_20e_in_basket_sessions,
SUM(reached_35e_in_basket_sessions) as total_reached_35e_in_basket_sessions,
SUM(reached_5th_product_in_cart_sessions) as total_reached_5th_product_in_cart_sessions,
FROM ga4_per_month
GROUP BY REGION,  CUSTOMER_ID),

ga4_aggregation as (Select
REGION,
CUSTOMER_ID,
avg(reached_5th_product_in_cart_sessions) as avg_reached_5th_product_in_cart_sessions,
avg(reached_35e_in_basket_sessions) as avg_reached_35e_in_basket_sessions,
avg(reached_20e_in_basket_sessions) as avg_reached_20e_in_basket_sessions,
avg(cart_addition_sessions) as avg_cart_addition_sessions,
avg(begin_checkout_sessions) as avg_begin_checkout_sessions,
avg(quality_sessions) as avg_quality_sessions
from ga4_per_month
group by REGION, CUSTOMER_ID
),

ga4_ratio as (SELECT
m.REGION,
m.CUSTOMER_ID,
m.SESSION_MONTH_PERIOD,
m.SESSION_YEAR_PERIOD,
number_days_since_last_session,
DIV0(reached_5th_product_in_cart_sessions, avg_reached_5th_product_in_cart_sessions) as ratio_period_5th_product_in_cart,
DIV0(reached_35e_in_basket_sessions, avg_reached_35e_in_basket_sessions) as ratio_reached_35e_in_basket,
DIV0(reached_20e_in_basket_sessions, avg_reached_20e_in_basket_sessions) as ratio_reached_20e_in_basket,
DIV0(cart_addition_sessions, avg_cart_addition_sessions) as ratio_cart_addition,
DIV0(begin_checkout_sessions, avg_begin_checkout_sessions) as ratio_begin_checkout,
DIV0(quality_sessions, avg_quality_sessions) as ratio_quality_sessions
from ga4_per_month m inner join ga4_aggregation a on a.CUSTOMER_ID=m.CUSTOMER_ID and a.REGION=m.REGION
),

nps_last_time as (
SELECT  REGION, CUSTOMER_INFORMATION_EMAIL, CUSTOMER_ID, MAX(CREATED_AT) as last_time
FROM ANALYTICS.DBT.FCT_ORDER
group by REGION,CUSTOMER_INFORMATION_EMAIL, CUSTOMER_ID
),

nps_last_score as (
SELECT  f.REGION,f.CUSTOMER_INFORMATION_EMAIL, f.CUSTOMER_ID, CUSTOMER_EFFORT_SCORE as last_NPS
FROM ANALYTICS.DBT.FCT_ORDER f inner join nps_last_time l on f.CUSTOMER_ID=l.CUSTOMER_ID and f.CREATED_AT=l.last_time and f.REGION=l.REGION and f.CUSTOMER_INFORMATION_EMAIL=l.CUSTOMER_INFORMATION_EMAIL
),

nps_average_score as(
SELECT  REGION,CUSTOMER_INFORMATION_EMAIL,CUSTOMER_ID, AVG(CUSTOMER_EFFORT_SCORE) as NPS_AVG_score
FROM ANALYTICS.DBT.FCT_ORDER
group by REGION,CUSTOMER_INFORMATION_EMAIL,CUSTOMER_ID
),

nps_data as(select
a.REGION,a.CUSTOMER_INFORMATION_EMAIL,a.CUSTOMER_ID, NPS_AVG_score, last_NPS
from nps_average_score a inner join nps_last_score l on a.CUSTOMER_ID=l.CUSTOMER_ID and a.REGION=l.REGION and a.CUSTOMER_INFORMATION_EMAIL=l.CUSTOMER_INFORMATION_EMAIL
order by REGION,CUSTOMER_ID),

--first calculation of the delivery method preference
by_delivery_method as (SELECT REGION,CUSTOMER_INFORMATION_EMAIL,
CUSTOMER_ID, SHIPPING_METHOD_NAME, count(distinct order_id) as number_order_by_delivery_method
FROM ANALYTICS.DBT.FCT_ORDER
GROUP BY 1,2,3,4),

total_order_delivery as (SELECT REGION,CUSTOMER_INFORMATION_EMAIL,
CUSTOMER_ID, count(distinct order_id) as number_orders
FROM ANALYTICS.DBT.FCT_ORDER
GROUP BY 1,2,3),

delivery_preference as (select
t.REGION,
t.CUSTOMER_INFORMATION_EMAIL,
t.CUSTOMER_ID,
SHIPPING_METHOD_NAME as Delivery_method_preference,
number_order_by_delivery_method/number_orders as ratio
from total_order_delivery t join by_delivery_method d on t.CUSTOMER_ID=d.CUSTOMER_ID  and t.REGION=d.REGION  and t.CUSTOMER_INFORMATION_EMAIL=d.CUSTOMER_INFORMATION_EMAIL
where ratio > 0.80),

row_order_data as (SELECT
 o.REGION,ORDER_ROW_ID, CUSTOMER_INFORMATION_EMAIL, CUSTOMER_ID, o.PRODUCT_ID, o.QUANTITY, MAIN_CATEGORY_NAME, MAIN_CATEGORY_ID
FROM ANALYTICS.DBT.FCT_ORDER_ROW o JOIN ANALYTICS.DBT.DIM_PRODUCT p
WHERE o.REGION=p.REGION AND o.PRODUCT_ID=p.PRODUCT_ID
ORDER BY REGIOn, CUSTOMER_INFORMATION_EMAIL, CUSTOMER_ID),
total_product_category as (SELECT REGION,CUSTOMER_INFORMATION_EMAIL,
    CUSTOMER_ID, SUM(QUANTITY) as total_cat, MAIN_CATEGORY_NAME, MAIN_CATEGORY_ID
    from row_order_data
    group by REGION,CUSTOMER_INFORMATION_EMAIL,CUSTOMER_ID,MAIN_CATEGORY_NAME, MAIN_CATEGORY_ID),
total_order_category as (select
    REGION,CUSTOMER_INFORMATION_EMAIL,CUSTOMER_ID, SUM(QUANTITY) as total_ordered
    from row_order_data
    group by REGION,CUSTOMER_INFORMATION_EMAIL,CUSTOMER_ID),
category_preference as (select
t.REGION,t.CUSTOMER_INFORMATION_EMAIL,t.CUSTOMER_ID, MAIN_CATEGORY_NAME as Special_diet, MAIN_CATEGORY_ID as Special_diet_id, total_cat/total_ordered as ratio
from total_order_category t join total_product_category c on t.CUSTOMER_ID=c.CUSTOMER_ID and t.REGION=c.REGION and t.CUSTOMER_INFORMATION_EMAIL=c.CUSTOMER_INFORMATION_EMAIL
where total_ordered>0 and ratio > 0.80 and MAIN_CATEGORY_ID!=0),

order_per_month as (SELECT
REGION,
CUSTOMER_INFORMATION_EMAIL,
CUSTOMER_ID,
YEAR(CREATED_AT) as SESSION_YEAR_PERIOD,
MONTH(CREATED_AT) as SESSION_MONTH_PERIOD,
COUNT(ORDER_ID) as NUMBER_ORDER
FROM ANALYTICS.DBT.FCT_ORDER
GROUP BY REGION, CUSTOMER_INFORMATION_EMAIL,CUSTOMER_ID, SESSION_YEAR_PERIOD,SESSION_MONTH_PERIOD),

order_total as(select
m.REGION,
m.CUSTOMER_INFORMATION_EMAIL,
m.CUSTOMER_ID,
SESSION_MONTH_PERIOD,
SESSION_YEAR_PERIOD,
NUMBER_ORDER_TOTAL,
DIV0(NUMBER_ORDER, NUMBER_ORDER_TOTAL) as order_frequency
FROM order_per_month m inner join order_aggregate a on a.CUSTOMER_ID=m.CUSTOMER_ID and m.REGION=a.REGION and m.CUSTOMER_INFORMATION_EMAIL=a.CUSTOMER_INFORMATION_EMAIL),

reference_ga4 as (select
r.REGION,
r.CUSTOMER_INFORMATION_EMAIL,
r.CUSTOMER_ID,
theMonth,
theMonthNum,
theYear,
number_days_since_last_session,
ratio_period_5th_product_in_cart,
ratio_reached_35e_in_basket,
ratio_reached_20e_in_basket,
ratio_cart_addition,
ratio_begin_checkout,
ratio_quality_sessions
FROM reference_list r LEFT OUTER JOIN ga4_ratio ga4 on ga4.CUSTOMER_ID=r.CUSTOMER_ID and ga4.SESSION_MONTH_PERIOD=r.theMonthNum and r.theYear=ga4.SESSION_YEAR_PERIOD and r.REGION=ga4.REGION),



pre_total_ces as (SELECT
c.REGION,
c.CUSTOMER_INFORMATION_EMAIL,
c.CUSTOMER_ID,
number_days_since_last_order,
AVERAGE_ORDER_TOTAL_INCLUDING_VAT,
LIFETIME_DISCOUNT_VALUE_VAT0,
LIFETIME_NET_SALES_VAT0,
DAYS_TO_REPURCHASE,
DAYS_SINCE_FIRST_ORDER,
ORDER_COUNT,
STATUS,
DIV0(last_CES, CES_AVG_score) as CES_last_vs_Average
FROM customer_row c LEFT OUTER JOIN ces_data on ces_data.CUSTOMER_ID=c.CUSTOMER_ID),

pre_total_ces_nps as (SELECT
c.REGION,
c.CUSTOMER_INFORMATION_EMAIL,
c.CUSTOMER_ID,
number_days_since_last_order,
AVERAGE_ORDER_TOTAL_INCLUDING_VAT,
LIFETIME_DISCOUNT_VALUE_VAT0,
LIFETIME_NET_SALES_VAT0,
DAYS_TO_REPURCHASE,
DAYS_SINCE_FIRST_ORDER,
ORDER_COUNT,
STATUS,
CES_last_vs_Average,
DIV0(last_NPS, NPS_AVG_score) as NPS_last_vs_Average
FROM pre_total_ces c LEFT OUTER JOIN nps_data on nps_data.CUSTOMER_ID=c.CUSTOMER_ID and c.REGION=nps_data.REGION and c.CUSTOMER_INFORMATION_EMAIL=nps_data.CUSTOMER_INFORMATION_EMAIL),

pre_total_ces_nps_delivery as (SELECT
c.REGION,
c.CUSTOMER_INFORMATION_EMAIL,
c.CUSTOMER_ID,
number_days_since_last_order,
AVERAGE_ORDER_TOTAL_INCLUDING_VAT,
LIFETIME_DISCOUNT_VALUE_VAT0,
LIFETIME_NET_SALES_VAT0,
DAYS_TO_REPURCHASE,
DAYS_SINCE_FIRST_ORDER,
ORDER_COUNT,
STATUS,
CES_last_vs_Average,
NPS_last_vs_Average,
Delivery_method_preference
FROM pre_total_ces_nps c LEFT OUTER JOIN delivery_preference d on d.CUSTOMER_ID=c.CUSTOMER_ID and c.REGION=d.REGION and c.CUSTOMER_INFORMATION_EMAIL=d.CUSTOMER_INFORMATION_EMAIL),

pre_total_ces_nps_delivery_category as (SELECT
c.REGION,
c.CUSTOMER_INFORMATION_EMAIL,
c.CUSTOMER_ID,
number_days_since_last_order,
AVERAGE_ORDER_TOTAL_INCLUDING_VAT,
LIFETIME_DISCOUNT_VALUE_VAT0,
LIFETIME_NET_SALES_VAT0,
DAYS_TO_REPURCHASE,
DAYS_SINCE_FIRST_ORDER,
ORDER_COUNT,
STATUS,
CES_last_vs_Average,
NPS_last_vs_Average,
Delivery_method_preference,
Special_diet
FROM pre_total_ces_nps_delivery c LEFT OUTER JOIN category_preference d on d.CUSTOMER_ID=c.CUSTOMER_ID and c.REGION=d.REGION and c.CUSTOMER_INFORMATION_EMAIL=d.CUSTOMER_INFORMATION_EMAIL),


pre_total_ces_nps_delivery_category_ga4 as (SELECT
c.REGION,
c.CUSTOMER_INFORMATION_EMAIL,
c.CUSTOMER_ID,
number_days_since_last_order,
AVERAGE_ORDER_TOTAL_INCLUDING_VAT,
LIFETIME_DISCOUNT_VALUE_VAT0,
LIFETIME_NET_SALES_VAT0,
DAYS_TO_REPURCHASE,
DAYS_SINCE_FIRST_ORDER,
ORDER_COUNT,
STATUS,
CES_last_vs_Average,
NPS_last_vs_Average,
Delivery_method_preference,
Special_diet,
number_days_since_last_session,
total_sessions,
total_engaged_sessions,
total_quality_sessions,
total_cart_addition_sessions,
total_begin_checkout_sessions,
total_subscribers_sessions,
total_reached_20e_in_basket_sessions,
total_reached_35e_in_basket_sessions,
total_reached_5th_product_in_cart_sessions,
FROM pre_total_ces_nps_delivery_category c LEFT OUTER JOIN ga4_total d on d.CUSTOMER_ID=c.CUSTOMER_ID and c.REGION=d.REGION ),

reference_ga4_april2024 as (select
ga4.REGION,
ga4.CUSTOMER_INFORMATION_EMAIL,
ga4.CUSTOMER_ID,
theMonth,
theMonthNum,
theYear,
ratio_period_5th_product_in_cart as ratio_042024_5th_product_in_cart,
ratio_reached_35e_in_basket as ratio_042024_reached_35e_in_basket,
ratio_reached_20e_in_basket as ratio_042024_reached_20e_in_basket,
ratio_cart_addition as ratio_042024_cart_addition,
ratio_begin_checkout as ratio_042024_begin_checkout,
ratio_quality_sessions as ratio_042024_quality_sessions
FROM reference_ga4 ga4
WHERE ga4.theMonthNum=04 and 2024=ga4.theYear),

reference_ga4_march2024 as (select
ga4.REGION,
ga4.CUSTOMER_INFORMATION_EMAIL,
ga4.CUSTOMER_ID,
theMonth,
theMonthNum,
theYear,
ratio_period_5th_product_in_cart as ratio_032024_5th_product_in_cart,
ratio_reached_35e_in_basket as ratio_032024_reached_35e_in_basket,
ratio_reached_20e_in_basket as ratio_032024_reached_20e_in_basket,
ratio_cart_addition as ratio_032024_cart_addition,
ratio_begin_checkout as ratio_032024_begin_checkout,
ratio_quality_sessions as ratio_032024_quality_sessions
FROM reference_ga4 ga4
WHERE ga4.theMonthNum=03 and 2024=ga4.theYear),

reference_ga4_february2024 as (select
ga4.REGION,
ga4.CUSTOMER_INFORMATION_EMAIL,
ga4.CUSTOMER_ID,
theMonth,
theMonthNum,
theYear,
ratio_period_5th_product_in_cart as ratio_022024_5th_product_in_cart,
ratio_reached_35e_in_basket as ratio_022024_reached_35e_in_basket,
ratio_reached_20e_in_basket as ratio_022024_reached_20e_in_basket,
ratio_cart_addition as ratio_022024_cart_addition,
ratio_begin_checkout as ratio_022024_begin_checkout,
ratio_quality_sessions as ratio_022024_quality_sessions
FROM reference_ga4 ga4
WHERE ga4.theMonthNum=02 and 2024=ga4.theYear),

pre_total_april2024 as(SELECT
o.REGION,
o.CUSTOMER_INFORMATION_EMAIL,
o.CUSTOMER_ID,
SESSION_MONTH_PERIOD,
SESSION_YEAR_PERIOD,
order_frequency as order_frequency_042024,
ratio_042024_quality_sessions,
ratio_042024_begin_checkout,
ratio_042024_cart_addition,
ratio_042024_reached_20e_in_basket,
ratio_042024_reached_35e_in_basket,
ratio_042024_5th_product_in_cart
FROM order_total o LEFT OUTER JOIN reference_ga4_april2024 ga4 on ga4.CUSTOMER_ID=o.CUSTOMER_ID AND o.SESSION_MONTH_PERIOD=ga4.theMonthNum  AND o.SESSION_YEAR_PERIOD=ga4.theYear and ga4.REGION=o.REGION and o.CUSTOMER_INFORMATION_EMAIL=ga4.CUSTOMER_INFORMATION_EMAIL
where SESSION_MONTH_PERIOD=04 and SESSION_YEAR_PERIOD=2024),

pre_total_march2024 as(SELECT
o.REGION,
o.CUSTOMER_INFORMATION_EMAIL,
o.CUSTOMER_ID,
SESSION_MONTH_PERIOD,
SESSION_YEAR_PERIOD,
order_frequency as order_frequency_032024,
ratio_032024_quality_sessions,
ratio_032024_begin_checkout,
ratio_032024_cart_addition,
ratio_032024_reached_20e_in_basket,
ratio_032024_reached_35e_in_basket,
ratio_032024_5th_product_in_cart
FROM order_total o LEFT OUTER JOIN reference_ga4_march2024 ga4 on ga4.CUSTOMER_ID=o.CUSTOMER_ID AND o.SESSION_MONTH_PERIOD=ga4.theMonthNum  AND o.SESSION_YEAR_PERIOD=ga4.theYear and ga4.REGION=o.REGION and o.CUSTOMER_INFORMATION_EMAIL=ga4.CUSTOMER_INFORMATION_EMAIL
where SESSION_MONTH_PERIOD=03 and SESSION_YEAR_PERIOD=2024 ),

pre_total_february2024 as(SELECT
o.REGION,
o.CUSTOMER_INFORMATION_EMAIL,
o.CUSTOMER_ID,
SESSION_MONTH_PERIOD,
SESSION_YEAR_PERIOD,
order_frequency as order_frequency_022024,
ratio_022024_quality_sessions,
ratio_022024_begin_checkout,
ratio_022024_cart_addition,
ratio_022024_reached_20e_in_basket,
ratio_022024_reached_35e_in_basket,
ratio_022024_5th_product_in_cart
FROM order_total o LEFT OUTER JOIN reference_ga4_february2024 ga4 on ga4.CUSTOMER_ID=o.CUSTOMER_ID AND o.SESSION_MONTH_PERIOD=ga4.theMonthNum  AND o.SESSION_YEAR_PERIOD=ga4.theYear  and ga4.REGION=o.REGION and o.CUSTOMER_INFORMATION_EMAIL=ga4.CUSTOMER_INFORMATION_EMAIL
where SESSION_MONTH_PERIOD=02 and SESSION_YEAR_PERIOD=2024),

pre_total_ces_nps_delivery_category_april as(SELECT
c.REGION,
c.CUSTOMER_INFORMATION_EMAIL,
c.CUSTOMER_ID,
number_days_since_last_order,
AVERAGE_ORDER_TOTAL_INCLUDING_VAT,
LIFETIME_DISCOUNT_VALUE_VAT0,
LIFETIME_NET_SALES_VAT0,
DAYS_TO_REPURCHASE,
DAYS_SINCE_FIRST_ORDER,
ORDER_COUNT,
STATUS,
CES_last_vs_Average,
NPS_last_vs_Average,
Delivery_method_preference,
Special_diet,
number_days_since_last_session,
total_sessions,
total_engaged_sessions,
total_quality_sessions,
total_cart_addition_sessions,
total_begin_checkout_sessions,
total_subscribers_sessions,
total_reached_20e_in_basket_sessions,
total_reached_35e_in_basket_sessions,
total_reached_5th_product_in_cart_sessions,
order_frequency_042024,
ratio_042024_quality_sessions,
ratio_042024_begin_checkout,
ratio_042024_cart_addition,
ratio_042024_reached_20e_in_basket,
ratio_042024_reached_35e_in_basket,
ratio_042024_5th_product_in_cart
FROM pre_total_ces_nps_delivery_category_ga4 c LEFT OUTER JOIN pre_total_april2024 d on d.CUSTOMER_ID=c.CUSTOMER_ID AND d.REGION=c.REGION and c.CUSTOMER_INFORMATION_EMAIL=d.CUSTOMER_INFORMATION_EMAIL),


pre_total_ces_nps_delivery_category_april_march as(SELECT
c.REGION,
c.CUSTOMER_INFORMATION_EMAIL,
c.CUSTOMER_ID,
number_days_since_last_order,
AVERAGE_ORDER_TOTAL_INCLUDING_VAT,
LIFETIME_DISCOUNT_VALUE_VAT0,
LIFETIME_NET_SALES_VAT0,
DAYS_TO_REPURCHASE,
DAYS_SINCE_FIRST_ORDER,
ORDER_COUNT,
STATUS,
CES_last_vs_Average,
NPS_last_vs_Average,
Delivery_method_preference,
Special_diet,
number_days_since_last_session,
total_sessions,
total_engaged_sessions,
total_quality_sessions,
total_cart_addition_sessions,
total_begin_checkout_sessions,
total_subscribers_sessions,
total_reached_20e_in_basket_sessions,
total_reached_35e_in_basket_sessions,
total_reached_5th_product_in_cart_sessions,
order_frequency_042024,
ratio_042024_quality_sessions,
ratio_042024_begin_checkout,
ratio_042024_cart_addition,
ratio_042024_reached_20e_in_basket,
ratio_042024_reached_35e_in_basket,
ratio_042024_5th_product_in_cart,
order_frequency_032024,
ratio_032024_quality_sessions,
ratio_032024_begin_checkout,
ratio_032024_cart_addition,
ratio_032024_reached_20e_in_basket,
ratio_032024_reached_35e_in_basket,
ratio_032024_5th_product_in_cart
FROM pre_total_ces_nps_delivery_category_april c LEFT OUTER JOIN pre_total_march2024 d on d.CUSTOMER_ID=c.CUSTOMER_ID AND d.REGION=c.REGION and c.CUSTOMER_INFORMATION_EMAIL=d.CUSTOMER_INFORMATION_EMAIL),


pre_total_ces_nps_delivery_category_april_march_february as(SELECT
c.REGION,
c.CUSTOMER_INFORMATION_EMAIL,
c.CUSTOMER_ID,
number_days_since_last_order,
AVERAGE_ORDER_TOTAL_INCLUDING_VAT,
LIFETIME_DISCOUNT_VALUE_VAT0,
LIFETIME_NET_SALES_VAT0,
DAYS_TO_REPURCHASE,
DAYS_SINCE_FIRST_ORDER,
ORDER_COUNT,
STATUS,
CES_last_vs_Average,
NPS_last_vs_Average,
Delivery_method_preference,
Special_diet,
number_days_since_last_session,
total_sessions,
total_engaged_sessions,
total_quality_sessions,
total_cart_addition_sessions,
total_begin_checkout_sessions,
total_subscribers_sessions,
total_reached_20e_in_basket_sessions,
total_reached_35e_in_basket_sessions,
total_reached_5th_product_in_cart_sessions,
order_frequency_042024,
ratio_042024_quality_sessions,
ratio_042024_begin_checkout,
ratio_042024_cart_addition,
ratio_042024_reached_20e_in_basket,
ratio_042024_reached_35e_in_basket,
ratio_042024_5th_product_in_cart,
order_frequency_032024,
ratio_032024_quality_sessions,
ratio_032024_begin_checkout,
ratio_032024_cart_addition,
ratio_032024_reached_20e_in_basket,
ratio_032024_reached_35e_in_basket,
ratio_032024_5th_product_in_cart,
order_frequency_022024,
ratio_022024_quality_sessions,
ratio_022024_begin_checkout,
ratio_022024_cart_addition,
ratio_022024_reached_20e_in_basket,
ratio_022024_reached_35e_in_basket,
ratio_022024_5th_product_in_cart
FROM pre_total_ces_nps_delivery_category_april_march c LEFT OUTER JOIN pre_total_february2024 d on d.CUSTOMER_ID=c.CUSTOMER_ID AND d.REGION=c.REGION and c.CUSTOMER_INFORMATION_EMAIL=d.CUSTOMER_INFORMATION_EMAIL)


select * from pre_total_ces_nps_delivery_category_april_march_february
order by REGION, CUSTOMER_ID

""")

# Fetch the results
results = cur.fetchall()

# Display the results
for row in results:
    print(row)

# Close the cursor and the connection
cur.close()
conn.close()

About BiqQuery, the connector is harder to put in place for only one extract. Therefore the extract is done separately, and the result is stored within GitHub.
The file query can be found here: https://github.com/Fiksuruoka-fi/Churn-prediction/blob/main/BigQuery-query.txt


In [ ]:
urlbe = 'https://github.com/Fiksuruoka-fi/Churn-prediction/blob/main/BE_bquxjob_3943529_18f7b2f60ea.csv'
dfbe = pd.read_csv(urlbe)
urlnl = 'https://github.com/Fiksuruoka-fi/Churn-prediction/blob/main/NL_bquxjob_bf1cfa8_18f7b2e6e41.csv'
dfnl = pd.read_csv(urlnl)
urlfi2 = 'https://github.com/Fiksuruoka-fi/Churn-prediction/blob/main/FI-2_bquxjob_5b86c44_18f7b35c300.csv'
dffi2 = pd.read_csv(urlfi2)
urlfi1 = 'https://github.com/Fiksuruoka-fi/Churn-prediction/blob/main/FI-1_bquxjob_3605090a_18f7b33e0c1.csv'
dffi1 = pd.read_csv(urlfi1)
# Dataset is now stored in a Pandas Dataframe

NameError: name 'pd' is not defined

Unfortunately BigQuery has some limitation when it comes to extract and save the data as CSV (except by using an extra BQ table which does not help in our case) So now we put all those extract from BigQuery together.

In [ ]:
# compile the list of dataframes you want to merge
data_frames = [dfbe, dfnl, dffi1, dffi2]
data_frames.rename(columns={"REGION": "REGION", "user_id": "CUSTOMER_ID", "campaign": "CAMPAIGN", "medium": "MEDIUM", "source": "SOURCE", "LP_benefit_activation":"LP_BENEFIT", "unsubscribe": "UNSUBSCRIBE", "cancellation": "CANCELLATION"})

Data preparation
We have the Snowflake query + the bigQuery extract. On top we are still missing Custobar extract.

In [ ]:
results.merge(data_frames, how='left', on='CUSTOMER_ID')

In [ ]:
# Importing pandas and seaborn libraries for data manipulation and charting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Creating a copy of the data frame to be used to
# correlate unpsuprvised learning labels with existing categorical variables
data_orig = data.copy()

In [ ]:
# Following data columns are not important for data analysis
# those have to be defined and are not yet fixed except "STATUS"
to_drop = ['STATUS']#,'REGION','CUSTOMER_INFORMATION_EMAIL','CUSTOMER_ID']
# Fix all columns
data = column_name_remove_space(data)
data.drop(columns=to_drop,inplace=True)

In [ ]:
# Check for null values
data.isna().any(axis=0).any()
# Replace null values by 0 as in our case that's what it means
data.fillna(0)
# Also we want to work with data that has value, so dropping all rows that have only NA
data = data.dropna(how='all')

In [ ]:
# Checking number of unique values in each column
data.nunique()
# Check all the unique values for all the columns having less than 100 unique values
# Avoiding the value prints for those columns which have large number of unique values like zipcode or counties etc.

for col in data.columns:
    if data[col].nunique()<100:
        print(col, sorted(data[col].unique()), '\n')


In [ ]:
# security check
# Check data by printing first few rows
data.head()
# Check data by printing last few rows
data.tail()

In [ ]:
# It is important to bring various columns or fields to the
# same scale because KMeans work based on distance between points
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(data)
scaled_data_array = sc.transform(data)
scaled_data = pd.DataFrame(scaled_data_array, columns = data.columns)
scaled_data.head()

In [ ]:
# Export data to CSV for submission purposes
scaled_data.to_csv('scaled_data.csv')

In [ ]:
# Running KMeans models to create chart for elbow method

from sklearn.cluster import KMeans
ks = range(1, 10)    # varying cluster values from 1 to 10
inertias = []

for k in ks:
    model = KMeans(n_clusters=k)
    model.fit(scaled_data)
    inertias.append(model.inertia_)

# Plot ks vs inertias, to apply elbow method
plt.plot(ks, inertias, '-o')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

There is a need for a break here to adjust with the Elbow method and know how many the cluster the method recommends.
Anyway we will also try with a fixed number of cluster that corresponds to the internal classification that we have define,
- Inactive customers: that group can be split into “inactive_1”, “inactive_2”,
“inactive_3plus”.
- Onboarding customers: those will be very hard to identify I guess as even on the concept of it, it's hard to explain.
- Reactivated customers: those will be very hard to identify I guess as even on the concept of it, it's hard to explain.
- Active customers: this group can also be split into “active_low_spend”, “active_medium_spend”, “active_high_spend”.

In [ ]:
# Based on elbow method, optimal number of clustering is ###
# Hence, creating KMeans model with ### clusters for further analysis

model = KMeans(n_clusters=###)
model.fit(scaled_data)
print(model.labels_)

In [ ]:
# Print cluster centers
model.cluster_centers_

In [ ]:
# Creating a new column in the original data frame containing KMeans labels

data_orig['label_'] = model.labels_

In [ ]:
# Exploring if KMeans labels have correlation with STATUS field
print(data_orig.query('label_==0 and STATUS=="####"').Customer_id.count())
print(data_orig.query('STATUS=="####"').Customer_id.count())

In [ ]:
# Under the assumption that the KMeans is being used as the predictive model for customer churn behavior
# we are not creating the confusion matrix

from sklearn import metrics
y_actual = data_orig['STATUS'].apply(lambda x: 1 if x=='No' else 0)
y_pred = data_orig['label_']
metrics.confusion_matrix(y_actual, y_pred)

In [ ]:
# And computing the accuracy score

metrics.accuracy_score(y_actual, y_pred)